<a href="https://colab.research.google.com/github/PikasXYZ/blending_mode_python/blob/main/blending_mode_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title blending modes
def apply_alpha(func): # a decorator to adjusting opacity
    def func_applied_alpha(top: np.array, bottom: np.array, alpha=1):
        assert 0 <= alpha <= 1, "alpha must be between 0 and 1 !"
        res = func(top, bottom)
        res = res * alpha + bottom * (1 - alpha)
        return res
    return func_applied_alpha

@apply_alpha
def normal(top: np.array, bottom: np.array) -> np.array: # 正常
    res = top
    res = res.astype(np.uint8)
    return res

# this blending mode isn't decorated by "apply_alpah" since its alpha is special
def dissolve(top: np.array, bottom: np.array, alpha=1) -> np.array: # 溶解
    sample = np.random.uniform(size=top.shape) > (1 - alpha)
    res = top * sample + bottom * (1 - sample)
    return res.astype(np.uint8)

@apply_alpha
def multiply(top: np.array, bottom: np.array) -> np.array: # 色彩增值
    top, bottom = top / 255.0, bottom / 255.0
    res = top * bottom
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def screen(top: np.array, bottom: np.array) -> np.array: # 濾色
    top, bottom = top / 255.0, bottom / 255.0
    res = 1 - (1 - top) * (1 - bottom)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def overlay(top: np.array, bottom: np.array) -> np.array: # 覆蓋
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(bottom < 0.5,
             2 * top * bottom,
             1 - 2 * (1 - top) * (1 - bottom))
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def soft_light(top: np.array, bottom: np.array) -> np.array: # 柔光
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(top < 0.5,
             (2 * top - 1) * (bottom - bottom**2) + bottom,
             (2 * top - 1) * (np.sqrt(bottom) - bottom) + bottom)
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def hard_light(top: np.array, bottom: np.array) -> np.array: # 實光
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(top < 0.5,
             2 * top * bottom,
             1 - 2 * (1 - top) * (1 - bottom))
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def color_dodge(top: np.array, bottom: np.array) -> np.array: # 加亮顏色
    top, bottom = top / 255.0, bottom / 255.0
    res = bottom / (1.0 - top + 0.001)  # prevent ZeroDivisionError
    res = np.clip(res, 0, 1)  # Ensures values are within [0, 1]
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def linear_dodge(top: np.array, bottom: np.array) -> np.array: # 線性加亮
    top, bottom = top / 255.0, bottom / 255.0
    res = np.clip(top + bottom, 0, 1)  # clips sum to [0, 1]
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def color_burn(top: np.array, bottom: np.array) -> np.array: # 加深顏色/顏色加深
    top, bottom = top / 255.0, bottom / 255.0
    res = 1.0 - (1.0 - bottom) / (top + 0.001)  # prevent ZeroDivisionError
    res = np.clip(res, 0, 1)  # Ensures values are within [0, 1]
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def linear_burn(top: np.array, bottom: np.array) -> np.array: # 線性加深
    top, bottom = top / 255.0, bottom / 255.0
    res = np.clip(top + bottom - 1.0, 0, 1)  # Sums, adjusts, and clips values to [0, 1]
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def vivid_light(top: np.array, bottom: np.array) -> np.array: # 強烈光源
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(bottom <= 0.5,
                   1 - (1 - top) / (2 * bottom + 0.001),  # prevent ZeroDivisionError
                   top / (2 * (1 - bottom) + 0.001)) # prevent ZeroDivisionError
    res = np.clip(res, 0, 1)
    res = (255 * res).astype(np.uint8)
    return res

@apply_alpha
def linear_light(top: np.array, bottom: np.array) -> np.array: # 線性光源
    top, bottom = top / 255.0, bottom / 255.0
    res = np.clip(2 * top + bottom - 1.0, 0, 1)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def lighten(top: np.array, bottom: np.array) -> np.array: # 變亮
    top, bottom = top / 255.0, bottom / 255.0
    res = np.maximum(top, bottom)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def darken(top: np.array, bottom: np.array) -> np.array: # 變暗
    top, bottom = top / 255.0, bottom / 255.0
    res = np.minimum(top, bottom)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def darker_color(top: np.array, bottom: np.array) -> np.array: # 顏色變暗
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(np.sum(top, axis=2, keepdims=True) < np.sum(bottom, axis=2, keepdims=True),
             top,
             bottom)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def lighter_color(top: np.array, bottom: np.array) -> np.array: # 顏色變亮
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(np.sum(top, axis=2, keepdims=True) > np.sum(bottom, axis=2, keepdims=True),
             top,
             bottom)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def pin_light(top: np.array, bottom: np.array) -> np.array: # 小光源
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(top <= 2 * bottom - 1,
             2 * bottom - 1,
             np.where(top >= 2 * bottom,
                  2 * bottom,
                  top))
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def hard_mix(top: np.array, bottom: np.array) -> np.array: # 實線疊印混合
    top, bottom = top / 255.0, bottom / 255.0
    res = np.where(top + bottom >= 1,
             1,
             0)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def divide(top: np.array, bottom: np.array) -> np.array:
    top, bottom = top / 255.0, bottom / 255.0
    res = np.clip(bottom / (top + 0.001), 0, 1)  # prevent ZeroDivisionError
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def difference(top: np.array, bottom: np.array) -> np.array: # 差異化
    top, bottom = top / 255.0, bottom / 255.0
    res = np.abs(bottom - top)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def exclusion(top: np.array, bottom: np.array) -> np.array: # 排除
    top, bottom = top / 255.0, bottom / 255.0
    res = (top + bottom) - 2 * top * bottom
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def subtract(top: np.array, bottom: np.array) -> np.array:
    top, bottom = top / 255.0, bottom / 255.0
    res = np.clip(bottom - top, 0, 1)
    res = (res * 255).astype(np.uint8)
    return res

@apply_alpha
def hue(top: np.array, bottom: np.array) -> np.array: # 色相
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    bottom_hsv = cv2.cvtColor(bottom, cv2.COLOR_BGR2HSV)
    bottom_hsv[:,:,0] = top_hsv[:,:,0]
    res = cv2.cvtColor(bottom_hsv, cv2.COLOR_HSV2BGR)
    return res

@apply_alpha
def saturation(top: np.array, bottom: np.array) -> np.array: # 飽和度
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    bottom_hsv = cv2.cvtColor(bottom, cv2.COLOR_BGR2HSV)
    bottom_hsv[:,:,1] = top_hsv[:,:,1]
    res = cv2.cvtColor(bottom_hsv, cv2.COLOR_HSV2BGR)
    return res

@apply_alpha
def color(top: np.array, bottom: np.array) -> np.array: # 顏色
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    bottom_hsv = cv2.cvtColor(bottom, cv2.COLOR_BGR2HSV)
    bottom_hsv[:,:,0] = top_hsv[:,:,0]
    bottom_hsv[:,:,1] = top_hsv[:,:,1]
    res = cv2.cvtColor(bottom_hsv, cv2.COLOR_HSV2BGR)
    return res

@apply_alpha
def luminosity(top: np.array, bottom: np.array) -> np.array: # 明度
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    bottom_hsv = cv2.cvtColor(bottom, cv2.COLOR_BGR2HSV)
    bottom_hsv[:,:,2] = top_hsv[:,:,2]
    res = cv2.cvtColor(bottom_hsv, cv2.COLOR_HSV2BGR)
    return res

blending_modes = {
    "normal": normal,
    "dissolve": dissolve,
    "multiply": multiply,
    "screen": screen,
    "overlay": overlay,
    "soft light": soft_light,
    "hard light": hard_light,
    "color dodge": color_dodge,
    "linear dodge": linear_dodge,
    "color burn": color_burn,
    "linear burn": linear_burn,
    "vivid light": vivid_light,
    "linear light": linear_light,
    "lighten": lighten,
    "darken": darken,
    "darker color": darker_color,
    "lighter color": lighter_color,
    "pin light": pin_light,
    "hard mix": hard_mix,
    "divide": divide,
    "difference": difference,
    "exclusion": exclusion,
    "subtract": subtract,
    "hue": hue,
    "saturation": saturation,
    "color": color,
    "luminosity": luminosity
}